# Naive Bayes Classifier
Neste notebook vamos testar a qualidade do algoritmo Naive Bayes para a classificação de nódulos

Vamos usar o <i><b>Bernoulli Naive Bayes</b></i> pois já tratamos os dados para estarem em formato binário e a classificação também é binária

In [ ]:
from sklearn.naive_bayes import BernoulliNB
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv('../data_cleaning/final_features.csv')

y = data["malignancy"]
X = data.drop(columns=['malignancy','case_id'])

## Hiperparâmetros

Vamos aplicar GridSearch com 10-fold cross-validation para testar o nosso modelo com diferentes hiperparâmetros.

No caso do Bernoulli Naive Bayes temos apenas a <i>suavização de Laplace</i> (atributo 'alpha') para variar, tendo em conta que é um algoritmo com poucos hiperparâmetros.

In [3]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold


# binarize = None pois os nossos dados ja estao em binario
# fit_prior = True vai permitir o algoritmo aprender as probabilidades de cada classe a priori
model = BernoulliNB(binarize=None, fit_prior=True)

# cross-validation para testar os hiperparametros
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=405)


print("Objetivo do teste com vários ranges: saber qual o melhor")
# hiperparametros a testar
param_grid = {
    'alpha': [0.01, 0.05, 0.1, 0.5, 1.0, 5.0],
}


# definir o GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='accuracy')

# treinar o modelo
grid_search.fit(X, y)

# resultado
print(f"Best Hyperparameters: {grid_search.best_params_}")

Objetivo do teste com vários ranges: saber qual o melhor
Best Hyperparameters: {'alpha': 0.01}


## Avaliação do Modelo

Após encontrar o melhor valor para os hiperparâmetros, vamos avaliar o modelo usando 10-fold cross-validation.

In [20]:
# métricas a utilizar para a avaliação do modelo
from sklearn.model_selection import cross_val_score
import numpy as np

# modelo
model = BernoulliNB(alpha=0.01, binarize=None, fit_prior=True)

# avaliação usando 10-fold cross validation
scores = [0,0,0,0] 
# [0] -> precision
# [1] -> f1
# [2] -> roc_auc
# [3] -> accuracy

scores[0] = (cross_val_score(model, X, y, cv=10, scoring = "precision"))
scores[1] = (cross_val_score(model, X, y, cv=10, scoring = "f1"))
scores[2] = (cross_val_score(model, X, y, cv=10, scoring = "roc_auc"))
scores[3] = (cross_val_score(model, X, y, cv=10, scoring = "accuracy"))


precision_scores_formatted = [f"{score:.2f}" for score in scores[0]]
print(f'Precision Scores: {precision_scores_formatted}')
F1_scores_formatted = [f"{score:.2f}" for score in scores[1]]
print(f'F1 Scores: {F1_scores_formatted}')
ROC_AUC_scores_formatted = [f"{score:.2f}" for score in scores[2]]
print(f'ROC_AUC Scores: {ROC_AUC_scores_formatted}')
Accuracy_scores_formatted = [f"{score:.2f}" for score in scores[3]]
print(f'Accuracy Scores: {Accuracy_scores_formatted}')

Precision Scores: ['0.71', '0.69', '0.72', '0.75', '0.72', '0.76', '0.72', '0.78', '0.77', '0.68']
F1 Scores: ['0.77', '0.76', '0.78', '0.82', '0.74', '0.76', '0.72', '0.77', '0.79', '0.74']
ROC_AUC Scores: ['0.88', '0.87', '0.89', '0.90', '0.86', '0.87', '0.87', '0.88', '0.89', '0.86']
Accuracy Scores: ['0.81', '0.79', '0.81', '0.84', '0.79', '0.81', '0.78', '0.82', '0.83', '0.78']


Os valores dos arrays dos scores acima serão utilizados para um teste estatístico (Wilcoxon Test) para averiguar se os modelos são estatisticamente diferentes.

Valor médio de cada métrica de avaliação do modelo:

In [21]:
print(f'Average Precision: {np.mean(scores[0]):.2f}')
print(f'F1 Score: {np.mean(scores[1]):.2f}')
print(f'ROC_AUC: {np.mean(scores[2]):.2f}')
print(f'Accuracy Score: {np.mean(scores[3]):.2f}')

Average Precision: 0.73
F1 Score: 0.76
ROC_AUC: 0.88
Accuracy Score: 0.81
